<a href="https://colab.research.google.com/github/Jason-Youu/unsloth-GRPO/blob/main/Qwen7binstruct_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [2]:
#%%caption let the output not shown
#--no-deps shows that dependence is not needed
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [3]:
#@title Colab Extra Install { display-mode: "form" }

%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [4]:
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jasonmichaels128 (jasonmichaels128-bar-ilan-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
wandb.init(project="GRPO-Qwen2.5-7b-instruct")

### Unsloth

Load up `Qwen/Qwen2.5-7B-Instruct`, and set parameters

In [6]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1800 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-7B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-15 19:55:39 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-15 19:55:39 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.4: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-7b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.43%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunke

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.5.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [8]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive')

#I run this model on the AIME dataset,
# with 8 completions for each question.
#Only those questions that are not all right or not all wrong are collected in datamath
from datasets import load_from_disk

!ls /content/drive/MyDrive/datasetmath/atasetmath/

dataset = load_from_disk('/content/drive/MyDrive/datasetmath')
print(dataset)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ls: cannot access '/content/drive/MyDrive/datasetmath/atasetmath/': No such file or directory
Dataset({
    features: ['ID', 'Year', 'Problem Number', 'Question', 'Answer', 'Part'],
    num_rows: 90
})


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [9]:
import re
from datasets import load_dataset, Dataset, load_from_disk

# Load and prep dataset
SYSTEM_PROMPT = """
You are a helpful math assistant
"""


# 解析模型输出，提取 box 中的内容
def extract_answer(text: str) -> str:
    match = re.search(r'\\boxed\{(\d+)\}', text)
    if match:
        return match.group(1)
    else:
        return None



def get_questions(dataset) -> Dataset:
    data = dataset.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['Question']}
        ],
        'answer': str(x["Answer"]).strip()
    }) # type: ignore
    return data # type: ignore
dataset = get_questions(dataset)
# Reward functions.

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Train the model
Now set up GRPO Trainer and all configurations!

In [10]:
max_prompt_length = 512

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 6,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 400,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

In [ ]:
#reward functions soft_format_reward_func, strict_format_reward_func,int_reward_func are deleted
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 90 | Num Epochs = 5 | Total steps = 400
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 80,740,352/7,000,000,000 (1.15% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


-------------------- Question:
A function $f$ is defined for all real numbers and satisfies $f(2+x)=f(2-x)$ and $f(7+x)=f(7-x)$ for all $x$ . If $x=0$ is a root for $f(x)=0$ , what is the least number of roots $f(x)=0$ must have in the interval $-1000\leq x \leq 1000$ ? 
Answer:
401 
Response:
To solve the problem, we need to analyze the properties of the function \( f \) given by the conditions \( f(2+x) = f(2-x) \) and \( f(7+x) = f(7-x) \). These conditions imply that \( f \) is symmetric about \( x = 2 \) and \( x = 7 \).

First, let's explore the symmetry properties:
1. \( f(2+x) = f(2-x) \) indicates that \( f \) is symmetric about \( x = 2 \).
2. \( f(7+x) = f(7-x) \) indicates that \( f \) is symmetric about \( x = 7 \).

These symmetries suggest that \( f \) is periodic. To find the period, we can use the fact that both symmetries must be consistent. Consider the transformation \( x \to x + 5 \):
\[ f(2 + (x+5)) = f(2 - (x+5)) \implies f(7 + x) = f(7 - x) \]
This is consistent

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / correctness_reward_func
1,0.000000,0.000000,0.000000,807.833374,0.000000,0.000000
2,0.000000,0.666667,1.032796,1222.333374,0.000000,0.666667
3,0.000000,0.333333,0.816497,531.166687,0.000004,0.333333
4,0.000000,1.000000,1.095445,735.166687,0.000004,1.000000
5,0.000000,0.000000,0.000000,794.666687,0.000005,0.000000
6,0.000000,1.000000,1.095445,887.500000,0.000004,1.000000
7,0.000000,0.000000,0.000000,749.666687,0.000004,0.000000
8,0.000000,0.000000,0.000000,826.333374,0.000002,0.000000
9,0.000000,0.000000,0.000000,980.833374,0.000003,0.000000
10,0.000000,0.000000,0.000000,960.833374,0.000003,0.000000


-------------------- Question:
The function $f$ defined by $f(x)= \frac{ax+b}{cx+d}$ , where $a$ , $b$ , $c$ and $d$ are nonzero real numbers, has the properties $f(19)=19$ , $f(97)=97$ and $f(f(x))=x$ for all values except $\frac{-d}{c}$ . Find the unique number that is not in the range of $f$ . 
Answer:
58 
Response:
To determine the unique number that is not in the range of the function \( f(x) = \frac{ax+b}{cx+d} \), we start by using the given properties of the function.

1. **Using \( f(19) = 19 \) and \( f(97) = 97 \):**

   \[
   f(19) = \frac{19a + b}{19c + d} = 19
   \]

   \[
   19a + b = 19(19c + d)
   \]

   \[
   19a + b = 361c + 19d \quad \text{(1)}
   \]

   \[
   f(97) = \frac{97a + b}{97c + d} = 97
   \]

   \[
   97a + b = 97(97c + d)
   \]

   \[
   97a + b = 9409c + 97d \quad \text{(2)}
   \]

2. **Subtract equation (1) from equation (2):**

   \[
   (97a + b) - (19a + b) = (9409c + 97d) - (361c + 19d)
   \]

   \[
   78a = 9048c + 78d
   \]

   \[
   a = 116c + d 

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / correctness_reward_func
1,0.000000,0.000000,0.000000,807.833374,0.000000,0.000000
2,0.000000,0.666667,1.032796,1222.333374,0.000000,0.666667
3,0.000000,0.333333,0.816497,531.166687,0.000004,0.333333
4,0.000000,1.000000,1.095445,735.166687,0.000004,1.000000
5,0.000000,0.000000,0.000000,794.666687,0.000005,0.000000
6,0.000000,1.000000,1.095445,887.500000,0.000004,1.000000
7,0.000000,0.000000,0.000000,749.666687,0.000004,0.000000
8,0.000000,0.000000,0.000000,826.333374,0.000002,0.000000
9,0.000000,0.000000,0.000000,980.833374,0.000003,0.000000
10,0.000000,0.000000,0.000000,960.833374,0.000003,0.000000


-------------------- Question:
Find the minimum value of $\frac{9x^2\sin^2 x + 4}{x\sin x}$ for $0 < x < \pi$ . 
Answer:
12 
Response:
To find the minimum value of the function \( f(x) = \frac{9x^2 \sin^2 x + 4}{x \sin x} \) for \( 0 < x < \pi \), we start by rewriting the function. Let \( y = x \sin x \). Then the function can be expressed as:

\[
f(x) = \frac{9x^2 \sin^2 x + 4}{x \sin x} = \frac{9y^2/x^2 + 4}{y} = \frac{9y^2/x^2 + 4}{y} = \frac{9y^2/x^2}{y} + \frac{4}{y} = \frac{9y}{x^2} + \frac{4}{y}
\]

Since \( y = x \sin x \), we need to express \( f(x) \) in terms of \( y \) and \( x \). Noting that \( y = x \sin x \), we can use the AM-GM inequality to find the minimum value. The function can be written as:

\[
f(x) = \frac{9x^2 \sin^2 x + 4}{x \sin x} = \frac{9 \left(\frac{y}{\sin x}\right) \sin^2 x + 4}{y/\sin x} = \frac{9 \cdot \frac{y}{\sin x} \cdot \sin^2 x + 4}{y/\sin x} = \frac{9y \sin x + 4}{y/\sin x} = 9 \sin x + \frac{4 \sin x}{y}
\]

Using the AM-GM inequality, we ha

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
#Example without any GRPO trained
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 2048,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.50s/it, est. speed input: 3.77 toks/s, output: 19.07 toks/s]


'Calculating the value of pi (π) to a high degree of precision can be a complex task, but I can provide you with an approximation. Pi is an irrational number, which means it has an infinite number of decimal places without repeating. Common approximations for everyday use include 3.14 or 22/7, but for more precise calculations, we can use more decimal places such as 3.141592653589793.\n\nIf you need a more precise value or a specific method to calculate it, please let me know! For instance, you might want to use a specific algorithm like the Monte Carlo method, the Bailey–Borwein–Plouffe (BBP) formula, or other series expansion methods.'

In [ ]:
# Load and prep dataset
# Evaluate the model without GRPO training
SYSTEM_PROMPT = """
You are a helpful math assistant
"""

def calculate_metrics(model, tokenizer, dataset_split):
    correct_predictions = 0
    total_predictions = 0

    sampling_params = SamplingParams(
        temperature=0.8,
        top_p=0.95,
        max_tokens=2048,
    )

    # Track accuracy per step
    step_accuracy = []

    for example in dataset_split:
        text = tokenizer.apply_chat_template([
            {"role" : "system", "content" : SYSTEM_PROMPT},
            {"role" : "user", "content" : example["prompt"][1]["content"]},
        ], tokenize = False, add_generation_prompt = True)

        # Generate model output
        output = model.fast_generate(
            text,
            sampling_params=sampling_params,
            lora_request= None,
        )[0].outputs[0].text
        print(output)
        # Extract response and compare with correct answer
        extracted_response = extract_answer(output)
        if extracted_response == example['answer']:
            correct_predictions += 1
        total_predictions += 1
        print(example['answer'])
        # Log per-step accuracy
        step_accuracy.append(correct_predictions / total_predictions if total_predictions > 0 else 0.0)

        # Optionally, log other metrics (e.g., output text, inputs)
        wandb.log({"step_accuracy_no": step_accuracy[-1]})

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0.0

    # Log overall accuracy to wandb
    wandb.log({"final_accuracy_no": accuracy})

    # Log a plot of accuracy progression over time
    wandb.log({
        "accuracy_plot": wandb.plot.line(
            xs=list(range(len(step_accuracy))),
            ys=step_accuracy,
            xlabel="Steps",
            ylabel="Accuracy",
            title="Accuracy Over Steps"
        )
    })

    return accuracy

In [ ]:
test_accuracy = calculate_metrics(model, tokenizer, dataset)
print(f"Test Accuracy: {test_accuracy:.4f}")














Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]












Processed prompts:  50%|█████     | 1/2 [00:14<00:14, 14.42s/it, est. speed input: 5.34 toks/s, output: 19.21 toks/s]












Processed prompts: 100%|██████████| 2/2 [00:21<00:00, 10.93s/it, est. speed input: 8.10 toks/s, output: 21.97 toks/s]


Let's break down the information step by step to solve the problem.

1. **Determine the number of sheep in Seattle**: We are given that Seattle has 20 sheep.

2. **Calculate the number of sheep in Charleston**: According to the problem, Charleston has 4 times as many sheep as Seattle. So, if Seattle has 20 sheep, then Charleston has:
   \[
   4 \times 20 = 80 \text{ sheep}
   \]

3. **Calculate the number of sheep in Toulouse**: The problem states that Toulouse has twice as many sheep as Charleston. Since Charleston has 80 sheep, Toulouse has:
   \[
   2 \times 80 = 160 \text{ sheep}
   \]

4. **Sum the number of sheep in all three cities**: We need to add the number of sheep in Toulouse, Charleston, and Seattle:
   \[
   160 + 80 + 20 = 260
   \]

<reasoning>
We used the given relationships between the number of sheep in each city to calculate the individual counts and then summed them up.
</reasoning>
<answer>
The total number of sheep in Toulouse, Charleston, and Seattle together is














Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.63s/it, est. speed input: 6.34 toks/s, output: 16.21 toks/s]


Let's break this down step by step.
...
</reasoning>
<answer>
3 bolts in total.
</answer>
...');
Let's break this down step by step.
First, we understand that a robe requires 2 bolts of blue fiber.
Next, we are told that the amount of white fiber needed is half that of the blue fiber.
Since the blue fiber is 2 bolts, half of that would be 1 bolt (because half of 2 is 1).
Therefore, the total amount of fiber needed for the robe is the sum of the blue and white fiber.
So, 2 bolts (blue) + 1 bolt (white) = 3 bolts in total.
</reasoning>
<answer>
3 bolts in total.
</answer>
3















Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:19<00:00, 19.97s/it, est. speed input: 4.61 toks/s, output: 17.03 toks/s]


...














Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.56s/it, est. speed input: 5.24 toks/s, output: 17.41 toks/s]


Let's break down the problem step by step to find the total number of pets that Jan, Marcia, and Cindy have.

1. First, we know that Cindy has 4 pets.
2. Marcia has two more pets than Cindy. Thus, Marcia has \(4 + 2 = 6\) pets.
3. Jan has three times the number of pets as Marcia. So, Jan has \(3 \times 6 = 18\) pets.

Now, to find the total number of pets, we add the pets of Jan, Marcia, and Cindy:

\[ \text{Total pets} = \text{Jan's pets} + \text{Marcia's pets} + \text{Cindy's pets} \]
\[ \text{Total pets} = 18 + 6 + 4 \]
\[ \text{Total pets} = 28 \]

<reasoning>
We established the number of pets each person has based on the given relationships and then summed them up to find the total.
</reasoning>
<answer>
The total number of pets is 28.
</answer>
28















Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]












Processed prompts: 100%|██████████| 1/1 [00:19<00:00, 19.44s/it, est. speed input: 4.42 toks/s, output: 17.49 toks/s]


Let's start by assigning variables to the ages of each child. Let:
- \( A \) be Amy's age,
- \( J \) be Jackson's age,
- \( C \) be Corey's age,
- \( M \) be James' age.

From the problem, we know:
1. Amy is 5 years older than Jackson: \( A = J + 5 \)
2. Amy is 2 years younger than Corey: \( A = C - 2 \)
3. James is 10 and is 1 year younger than Corey: \( M = 10 \) and \( M = C - 1 \)

From the information about James:
\[ M = 10 \]
\[ 10 = C - 1 \]
\[ C = 11 \]

Now we know Corey's age is 11. Using this in the second piece of information:
\[ A = C - 2 \]
\[ A = 11 - 2 \]
\[ A = 9 \]

Amy's age is 9. Now, using the first piece of information:
\[ A = J + 5 \]
\[ 9 = J + 5 \]
\[ J = 4 \]

Jackson's age is 4.

<reasoning>
We used the relationships given in the problem to set up and solve a system of equations. We first found Corey's age using the relationship with James, then used Corey's age to find Amy's age, and finally used Amy's age to find Jackson's age.
</reasoning>
<answer>
Jackson














Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

Now we load the LoRA and test:

In [ ]:
#Example after training
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("/content/lora model"),
)[0].outputs[0].text

output


Processed prompts: 100%|██████████| 1/1 [00:17<00:00, 17.66s/it, est. speed input: 2.15 toks/s, output: 16.03 toks/s]


"Calculating pi to a high degree of accuracy is a complex task typically done using computers and specific algorithms. However, I can demonstrate a simple way to approximate pi using a well-known method called the Leibniz formula for π:\n\n\\[\n\\pi = 4 \\times \\left(1 - \\frac{1}{3} + \\frac{1}{5} - \\frac{1}{7} + \\frac{1}{9} - \\cdots \\right)\n\\]\n\nThis is an infinite series that converges to π. Let's approximate π by summing the first 10000 terms of this series:\n\n\\[\n\\pi \\approx 4 \\times \\sum_{n=0}^{9999} \\frac{(-1)^n}{2n+1}\n\\]\n\nUsing a computational tool or programming language to sum these terms, we can approximate π. For the sake of this explanation, let's assume we have computed this sum:\n\n\\[\n\\pi \\approx 3.141592653589793\n\\]\n\nThis is a very close approximation to the actual value of π, which is an irrational number with an infinite, non-repeating decimal expansion.\n\n<answer>\n3.141592653589793\n</answer>\n"

In [ ]:
wandb.init(project="GRPO-Qwen7b-instruct")

step_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▇█
step_accuracy1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
step_accuracy,0.75
step_accuracy1,0.5


In [ ]:
# Evaluate the model with GRPO training (using math500 test set)
# I will evaluate the origianl model and the trainded model through Math-500

def get_math_questions() -> Dataset:
    data = load_dataset("gneubig/aime-1983-2024", split="train")# type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['problem']}
        ],
        'answer': str(x["Answer"]).strip()
    }) # type: ignore
    return data # type: ignore
datasetmath = get_math_questions()

In [ ]:
# Evaluate the model with GRPO training (using math500 test set)
def calculate_metrics_lora(model, tokenizer, dataset_split):
    correct_predictions = 0
    total_predictions = 0

    sampling_params = SamplingParams(
        temperature=0.8,
        top_p=0.95,
        max_tokens=2048,
    )

    # Track accuracy per step
    step_accuracy = []

    for example in dataset_split:
        text = tokenizer.apply_chat_template([
            {"role" : "system", "content" : SYSTEM_PROMPT},
            {"role" : "user", "content" : example["prompt"][1]["content"]},
        ], tokenize = False, add_generation_prompt = True)

        # Generate model output
        output = model.fast_generate(
            text,
            sampling_params=sampling_params,
            lora_request=model.load_lora("/content/lora model"),
        )[0].outputs[0].text
        print(output)
        # Extract response and compare with correct answer
        extracted_response = extract_answer(output)
        if extracted_response == example['answer']:
            correct_predictions += 1
        total_predictions += 1
        print(example['answer'])
        # Log per-step accuracy
        step_accuracy.append(correct_predictions / total_predictions if total_predictions > 0 else 0.0)

        # Optionally, log other metrics (e.g., output text, inputs)
        wandb.log({"step_accuracy": step_accuracy[-1]})

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0.0

    # Log overall accuracy to wandb
    wandb.log({"final_accuracy": accuracy})

    # Log a plot of accuracy progression over time
    wandb.log({
        "accuracy_plot": wandb.plot.line(
            xs=list(range(len(step_accuracy))),
            ys=step_accuracy,
            xlabel="Steps",
            ylabel="Accuracy",
            title="Accuracy Over Steps"
        )
    })

    return accuracy

# 评估模型
test_accuracy = calculate_metrics_lora(model, tokenizer, datasetmath[500:600])
print(f"Test Accuracy: {test_accuracy:.4f}")










Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]






Processed prompts:  50%|█████     | 1/2 [00:05<00:05,  5.27s/it, est. speed input: 27.69 toks/s, output: 66.76 toks/s]






Processed prompts: 100%|██████████| 2/2 [00:14<00:00,  7.42s/it, est. speed input: 16.58 toks/s, output: 33.90 toks/s]


To determine the number of cups of feed Wendi needs to give her chickens in the final meal of the day, we can follow these steps:

1. **Calculate the total daily feed requirement:**
   Each chicken needs 3 cups of feed per day. With 20 chickens, the total daily feed requirement is:
   \[
   3 \, \text{cups/chicken} \times 20 \, \text{chickens} = 60 \, \text{cups}
   \]

2. **Sum the feed given in the morning and afternoon:**
   In the morning, Wendi gives 15 cups of feed, and in the afternoon, she gives 25 cups of feed. The total feed given in these two meals is:
   \[
   15 \, \text{cups} + 25 \, \text{cups} = 40 \, \text{cups}
   \]

3. **Determine the remaining feed needed for the final meal:**
   Subtract the amount of feed already given from the total daily requirement:
   \[
   60 \, \text{cups} - 40 \, \text{cups} = 20 \, \text{cups}
   \]

Thus, Wendi needs to give her chickens 20 cups of feed in the final meal of the day.

<reasoning>
The problem involves calculating the total








Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]






Processed prompts: 100%|██████████| 1/1 [00:11<00:00, 11.44s/it, est. speed input: 5.34 toks/s, output: 15.74 toks/s]


<reasoning>
The problem states that a robe takes 2 bolts of blue fiber and half that amount in white fiber. 

First, we identify the amount of blue fiber needed: 2 bolts.

Next, we calculate the amount of white fiber needed, which is half the amount of blue fiber:
\[ \text{White fiber} = \frac{1}{2} \times \text{Blue fiber} = \frac{1}{2} \times 2 = 1 \text{ bolt} \]

Then, we add the bolts of blue and white fiber together to find the total amount of fiber needed for the robe:
\[ \text{Total fiber} = \text{Blue fiber} + \text{White fiber} = 2 + 1 = 3 \text{ bolts} \]

</reasoning>
<answer>
3
</answer>

3









Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]






Processed prompts: 100%|██████████| 1/1 [00:17<00:00, 17.26s/it, est. speed input: 5.33 toks/s, output: 15.59 toks/s]


...








Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]






Processed prompts: 100%|██████████| 1/1 [00:16<00:00, 16.42s/it, est. speed input: 7.98 toks/s, output: 16.02 toks/s]


Let's denote the number of cars that drove through the traffic jam in the first 15 minutes as \(x\).

Given:
- There are originally 30 cars.
- 5 cars take an exit, so \(30 - 5 = 25\) cars remain after some have taken an exit.
- In the second 15 minutes, 20 more cars drive through the traffic jam.

From the above, we can set up the equation for the total number of cars that drove through the traffic jam:
\[x + 20 = 25\]

To find \(x\), we solve the equation:
\[x = 25 - 20\]
\[x = 5\]

So, 5 cars drove through the traffic jam in the first 15 minutes.

<reasoning>
The original number of cars minus those that took an exit equals the number of cars left in the traffic jam. The sum of the cars that left in the first 15 minutes and the cars that left in the second 15 minutes equals the number of cars left in the traffic jam. We can then solve for the number of cars that left in the first 15 minutes.
</reasoning>
<answer>
5
</answer>

5









Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]






Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.81s/it, est. speed input: 13.29 toks/s, output: 15.79 toks/s]

<reasoning>
Mary currently has 2 potted plants on each of the 40 window ledges, which gives her a total of \(2 \times 40 = 80\) potted plants. If she decides to give away 1 potted plant from each ledge, she will give away \(1 \times 40 = 40\) potted plants. She also received 18 new potted plants. Therefore, after giving away 40 plants, she will have \(80 + 18 - 40 = 58\) potted plants remaining.
</reasoning>
<answer>
58
</answer>

58


TypeError: line() got an unexpected keyword argument 'xs'